E18 hcs crossing

In [ ]:
from scipy.io import readsav
import cdflib
import pyspedas
from pyspedas import time_string, time_double, tinterpol
import pytplot
from pytplot import tplot, store_data, get_data ,tlimit,xlim,ylim,tplot_options,options,split_vec,cdf_to_tplot,divide,tplot_names,get_timespan, tplot_rename, time_datetime
#import wget
import warnings
from warnings import simplefilter 
simplefilter(action='ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore")

import cdflib
from datetime import datetime,timedelta
import os.path
import bisect
import csv
from scipy import interpolate
import pandas as pd
import astropy.units as u,astropy.constants as const

#%matplotlib notebook
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
#import math functions 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import dateutil.parser

from matplotlib import ticker, cm
import warnings 
warnings.filterwarnings("ignore")


from warnings import simplefilter 
simplefilter(action='ignore', category=DeprecationWarning)

Import data from cdf file to tplot

In [ ]:
#protons
# spi_sf00_cdf = '/home/jovyan/data/cache/psp/data/sci/sweap/spi/L3/spi_sf00/2023/12/psp_swp_spi_sf00_L3_mom_20231229_v04.cdf'
spi_sf00_cdf = '/Users/robertalexander/Dropbox/__Collaborations/_NASA/__MH_Investigation/Downloaded_PSP_Data/High_Resolution_Protons/psp_swp_spi_af00_L3_mom_20230928_v04.cdf'

spi_sf00_tplot = cdf_to_tplot(spi_sf00_cdf,get_support_data=True)

#note both spi and spe vars import variables with same name when not using the pyspedas load routine, 
#so, need to add prefixes to distinguish them
for name in spi_sf00_tplot:
    tplot_rename(name, 'psp_spi_' + name)

In [ ]:
#electron pads
# spe_sf0_pad_cdf = '/home/jovyan/data/cache/psp/data/sci/sweap/spe/L3/spe_sf0_pad/2023/12/psp_swp_spe_sf0_L3_pad_20231229_v04.cdf'
spe_sf0_pad_cdf = '/Users/robertalexander/Dropbox/__Collaborations/_NASA/__MH_Investigation/Downloaded_PSP_Data/High_Resolution_Electrons/psp_swp_spe_af0_L3_pad_20230928_v04.cdf'
spe_sf0_pad_tplot = cdf_to_tplot(spe_sf0_pad_cdf,get_support_data=True)

for name in spe_sf0_pad_tplot:
    tplot_rename(name, 'psp_spe_' + name)

In [ ]:
# fld_mag_rtn_4sa_cdf = '/home/jovyan/data/cache/psp/data/sci/fields/l2/mag_RTN_4_Sa_per_Cyc/2023/12/psp_fld_l2_mag_RTN_4_Sa_per_Cyc_20231229_v02.cdf'
fld_mag_rtn_4sa_cdf = '/Users/robertalexander/Dropbox/__Collaborations/_NASA/__MH_Investigation/Downloaded_PSP_Data/Mag_RTN_4_Sa_per_Sec/psp_fld_l2_mag_RTN_4_Sa_per_Cyc_20230928_v02.cdf'

fld_mag_rtn_4sa_tplot = cdf_to_tplot(fld_mag_rtn_4sa_cdf,get_support_data=True)

In [ ]:
#This is the density from the Quasi-Thermal Noise, derived from the FIELD's Antennae, KEEP for comparison
# fld_sqtn_rfs_V1V2_cdf = '/home/jovyan/data/cache/psp/data/sci/fields/l3/sqtn_rfs_V1V2/2023/12/psp_fld_l3_sqtn_rfs_V1V2_20231229_v2.0.cdf'
fld_sqtn_rfs_V1V2_cdf = '/Users/robertalexander/Dropbox/__Collaborations/_NASA/__MH_Investigation/Downloaded_PSP_Data/FLDS_SQTN_RFS/psp_fld_l3_sqtn_rfs_V1V2_20230928_v2.0.cdf'

cdf_to_tplot(fld_sqtn_rfs_V1V2_cdf,get_support_data=True)

Open sf00 fit csv file

In [ ]:
date = '2023-09-28'
df_sf00 = pd.read_csv('spp_swp_spi_sf00_' + date + '.csv')
filtered_val_index = np.where((df_sf00['np1'] > 10) & (df_sf00['np2'] > 10) & (df_sf00['Tperp1'] > .01) & (df_sf00['Tperp2'] > .01) & (df_sf00['Trat1'] > .01) & (df_sf00['Trat2'] > .01) & (df_sf00['Trat1'] != 30)  & (df_sf00['Trat2'] != 30) & (df_sf00['Trat1'] != 2.0) & (df_sf00['Trat2'] != 2.0)& (df_sf00['Trat1'] != 1.0) & (df_sf00['Trat2'] != 1.0) & (df_sf00['vdrift'] != 10000) & (df_sf00['vdrift'] != -10000)) 
df_sf00 = df_sf00.iloc[filtered_val_index]
df_sf00['datetime'] = [datetime.utcfromtimestamp(t) for t in df_sf00.time]

In [ ]:
#define functions
def resample(data, times, new_times):
    ###interpolate data to times from data2
    interpol_f = interpolate.interp1d(times, data,fill_value="extrapolate")
    new_data1 = interpol_f(new_times)    
    return new_times, new_data1

def convert_time(start, stop):
    #input start and stop times as 'yyyy-mm-dd/hh:mm:ss'
    #return dataframe at those indices
    tindex_sf00 = np.where((df_sf00['datetime'] > start) & (df_sf00['datetime'] < stop))
    return df_sf00.iloc[tindex_sf00]

def time_clip(tarray,start,stop):
    #input time array in unix time format
    #input start and stop times as 'yyyy-mm-dd/hh:mm:ss'
    #return indices
    ind = np.logical_and(tarray>= time_double(start),tarray <= time_double(stop))
    return ind

#instability thresholds on brazil plot
beta_par       = np.arange(0, 1000, 0.02)
trat_parfire = 1-(.47/(beta_par - .59)**.53)
trat_oblfire = 1-(1.4/beta_par + .11)
trat_protcyc = 1+(.43/beta_par + .0004)
trat_mirror = 1+(.77/beta_par + .76 + .016)



#==============================================================
#==============================================================
#Access Downloaded data and define variables
#==============================================================
#==============================================================


#==============================================================
#Electrons
#==============================================================

#Access electron strahl pitch-angle distribution (PAD)
epad_data=get_data('psp_spe_EFLUX_VS_PA_E')
epad_PA = get_data('psp_spe_PITCHANGLE')

epad_times=epad_data.times
epad_vals = epad_data.y
epad_PA_vals = epad_PA.y

#Note for E1-E9, electron strahl energy bin is 8, for E10 and above, it's 12
energy_index = 12
epad_strahl = epad_vals[:,:,energy_index]

datetime_spe = time_datetime(epad_times)
times_spe_repeat = np.repeat(np.expand_dims(datetime_spe,1),12,1)

#compute centroids
centroids_epad = np.ma.average(epad_PA_vals, 
                       weights=epad_strahl, 
                       axis=1)
#interpolate to sf00 fit file cadence
estrahl_pad_centroid_spi_t, estrahl_pad_centroid_spi_ds = resample(centroids_epad, epad_times, df_sf00['time'])
#append to sf00 dataframe
df_sf00['estrahl_pad_centroid'] = estrahl_pad_centroid_spi_ds


#==============================================================
#Protons
#==============================================================

#Access components of velocity
split_vec('psp_spi_VEL_RTN_SUN')
vr = get_data('psp_spi_VEL_RTN_SUN_x')
vt = get_data('psp_spi_VEL_RTN_SUN_y')
vn = get_data('psp_spi_VEL_RTN_SUN_z')
vmag = np.sqrt(vr.y**2 + vt.y **2 + vn.y**2)
#interpolate to sf00 fit file cadence
vmag_spi_t, vmag_spi_ds = resample(vmag, vr.times, df_sf00['time'])
df_sf00['vsw_mom'] = vmag_spi_ds

#access proton energy flux vs energy
spi_eflux_v_energy = get_data('psp_spi_EFLUX_VS_ENERGY')
spi_eflux_v_theta = get_data('psp_spi_EFLUX_VS_THETA')
spi_eflux_v_phi = get_data('psp_spi_EFLUX_VS_PHI')

spi_times = spi_eflux_v_energy.times
spi_nrg_flux = spi_eflux_v_energy.y
spi_nrg_vals = spi_eflux_v_energy.v
spi_nrg_flux_theta = spi_eflux_v_theta.y
spi_nrg_vals_theta = spi_eflux_v_theta.v
spi_nrg_flux_phi = spi_eflux_v_phi.y
spi_nrg_vals_phi = spi_eflux_v_phi.v

datetime_spi = time_datetime(spi_times)
times_spi_repeat = np.repeat(np.expand_dims(datetime_spi,1),32,1)
times_spi_repeat_angle = np.repeat(np.expand_dims(datetime_spi,1),8,1)

#compute centroids
centroids_spi_nrg = np.ma.average(spi_nrg_vals, 
                       weights=spi_nrg_flux, 
                       axis=1)
centroids_spi_theta = np.ma.average(spi_nrg_vals_theta, 
                       weights=spi_nrg_flux_theta, 
                       axis=1)
centroids_spi_phi = np.ma.average(spi_nrg_vals_phi, 
                       weights=spi_nrg_flux_phi, 
                       axis=1)
#interpolate to sf00 fit file cadence
spi_nrg_centroid_spi_t, spi_nrg_centroid_spi_ds = resample(centroids_spi_nrg, spi_times, df_sf00['time'])
spi_theta_centroid_spi_t, spi_theta_centroid_spi_ds = resample(centroids_spi_theta, spi_times, df_sf00['time'])
spi_phi_centroid_spi_t, spi_phi_centroid_spi_ds = resample(centroids_spi_phi, spi_times, df_sf00['time'])
#append to sf00 dataframe
df_sf00['spi_nrg_centroid'] = spi_nrg_centroid_spi_ds
df_sf00['spi_theta_centroid'] = spi_theta_centroid_spi_ds
df_sf00['spi_phi_centroid'] = spi_phi_centroid_spi_ds


#==============================================================
#Mag
#==============================================================

#Access components of magentic field
split_vec('psp_fld_l2_mag_RTN_4_Sa_per_Cyc')
br_4sa = get_data('psp_fld_l2_mag_RTN_4_Sa_per_Cyc_x')
bt_4sa = get_data('psp_fld_l2_mag_RTN_4_Sa_per_Cyc_y')
bn_4sa = get_data('psp_fld_l2_mag_RTN_4_Sa_per_Cyc_z')
bmag_rtn4sa = np.sqrt(br_4sa.y**2 + bt_4sa.y **2 + bn_4sa.y**2)
time_b4sa = time_datetime(br_4sa.times)
#interpolate to sf00 fit file cadence
bmag_spi_t, bmag_spi_ds = resample(bmag_rtn4sa, br_4sa.times, df_sf00['time'])
df_sf00['Bmag'] = bmag_spi_ds #for e17, use this for Bmag 

split_vec('psp_spi_MAGF_INST')
binstx = get_data('psp_spi_MAGF_INST_x')
binsty = get_data('psp_spi_MAGF_INST_y')
binstz = get_data('psp_spi_MAGF_INST_z')
#interpolate to sf00 fit file cadence
binstx_spi_t, binstx_spi_ds = resample(binstx.y, binstx.times, df_sf00['time'])
binsty_spi_t, binsty_spi_ds = resample(binsty.y, binstx.times, df_sf00['time'])
binstz_spi_t, binstz_spi_ds = resample(binstz.y, binstx.times, df_sf00['time'])

df_sf00['Binst_x'] = binstx_spi_ds
df_sf00['Binst_y'] = binsty_spi_ds
df_sf00['Binst_z'] = binstz_spi_ds



#==============================================================
# sf00 fits
#==============================================================
#calculate variables
df_sf00['vp1_mag'] = np.sqrt(df_sf00['vp1_x']**2 + df_sf00['vp1_y']**2 + df_sf00['vp1_z']**2)
df_sf00['B_mag'] = np.sqrt(df_sf00['B_inst_x']**2 + df_sf00['B_inst_y']**2 + df_sf00['B_inst_z']**2)
df_sf00['bhat_x'] = df_sf00['B_inst_x']/df_sf00['B_mag']
df_sf00['bhat_y'] = df_sf00['B_inst_y']/df_sf00['B_mag']
df_sf00['bhat_z'] = df_sf00['B_inst_z']/df_sf00['B_mag']
m = 1836 * 511000.0 / (299792.0**2) #proton mass
df_sf00['n_tot'] = df_sf00['np1'] + df_sf00['np2']
df_sf00['Tperp_tot'] = (df_sf00['np1'] * df_sf00['Tperp1'] + df_sf00['np2'] * df_sf00['Tperp2'])/df_sf00['n_tot']
df_sf00['Tpar1'] = df_sf00['Tperp1']/df_sf00['Trat1']
df_sf00['Tpar2'] = df_sf00['Tperp2']/df_sf00['Trat2']
df_sf00['Tpar_tot'] = (df_sf00['np1'] * df_sf00['Tpar1'] + df_sf00['np2'] * df_sf00['Tpar2'] + (df_sf00['np1'] * df_sf00['np2'] / df_sf00['n_tot'])* m * (df_sf00['vdrift']**2))/df_sf00['n_tot'] 
df_sf00['Trat_tot'] = df_sf00['Tperp_tot']/df_sf00['Tpar_tot']

df_sf00['vcm_x'] = df_sf00['vp1_x'] + df_sf00['np1']/df_sf00['np2'] * df_sf00['vdrift'] * df_sf00['bhat_x'] # fitted center of mass velocity (x)
df_sf00['vcm_y'] = df_sf00['vp1_y'] + df_sf00['np1']/df_sf00['np2'] * df_sf00['vdrift'] * df_sf00['bhat_y'] # fitted center of mass velocity (y)
df_sf00['vcm_z'] = df_sf00['vp1_z'] + df_sf00['np1']/df_sf00['np2'] * df_sf00['vdrift'] * df_sf00['bhat_z'] # fitted center of mass velocity (z)
df_sf00['vcm_mag'] = np.sqrt(df_sf00['vcm_x']**2 + df_sf00['vcm_y']**2 + df_sf00['vcm_y']**2)

df_sf00['vp2_x'] = df_sf00['vp1_x'] + df_sf00['vdrift'] * df_sf00['bhat_x'] # fitted center of p2 velocity (x)
df_sf00['vp2_y'] = df_sf00['vp1_y'] + df_sf00['vdrift'] * df_sf00['bhat_y'] # fitted center of p2 velocity (y)
df_sf00['vp2_z'] = df_sf00['vp1_z'] + df_sf00['vdrift'] * df_sf00['bhat_z'] # fitted center of p2 velocity (x)
df_sf00['vp2_mag'] = np.sqrt(df_sf00['vp2_x']**2 + df_sf00['vp2_y']**2 + df_sf00['vp2_z']**2)

df_sf00['vcm_y'] = df_sf00['vp1_y'] + df_sf00['np1']/df_sf00['np2'] * df_sf00['vdrift'] * df_sf00['bhat_y'] # fitted center of mass velocity (y)
df_sf00['vcm_z'] = df_sf00['vp1_z'] + df_sf00['np1']/df_sf00['np2'] * df_sf00['vdrift'] * df_sf00['bhat_z'] # fitted center of mass velocity (z)
df_sf00['vcm_mag'] = np.sqrt(df_sf00['vcm_x']**2 + df_sf00['vcm_y']**2 + df_sf00['vcm_z']**2)

#compute heat flux
m = 1836 * 511000.0 / (299792.0**2)

#convert to thermal speeds
vt1perp2 = 2 * df_sf00['Tperp1']/m 
vt2perp2 = 2 * df_sf00['Tperp2']/m 
vt1par2 = 2 * df_sf00['Tpar1']/m 
vt2par2 = 2 * df_sf00['Tpar2']/m 

fac = 1.602E-10 #W/m2 conversion
df_sf00['qz_p'] = fac * 0.5 * m * ((df_sf00['np1'] * df_sf00['np2']) / df_sf00['n_tot']) * df_sf00['vdrift'] * (1.5 * (vt2par2 - vt1par2) + df_sf00['vdrift']**2 * (df_sf00['np1']**2 - df_sf00['np2']**2)/df_sf00['n_tot']**2 + (vt2perp2 - vt1perp2))
#compute normalisation factors

df_sf00['vt_perp_tot'] = np.sqrt(2 * (df_sf00['np1'] * df_sf00['Tperp1'] + df_sf00['np2'] * df_sf00['Tperp2']) / (m * df_sf00['n_tot']))
df_sf00['vt_par_tot'] = np.sqrt(2 * (df_sf00['np1'] * df_sf00['Tpar1']+ df_sf00['np2'] * df_sf00['Tpar2'] + m * df_sf00['vdrift']**2 * (df_sf00['np1'] * df_sf00['np2'])/df_sf00['n_tot']) / (m * df_sf00['n_tot']))

df_sf00['qz_p_perp']= df_sf00['qz_p'] / (m * df_sf00['n_tot'] * df_sf00['vt_perp_tot']**3)    # also normalise by mass
df_sf00['qz_p_par'] = df_sf00['qz_p'] / (m * df_sf00['n_tot'] * df_sf00['vt_par_tot']**3)

#Compute Alfven Mach number and Proton Parallel, Perp Beta

#calculate Alfven speed
df_sf00['valfven_pfits'] = 21.8 * df_sf00['B_mag']/ np.sqrt(df_sf00['n_tot']) #using proton p1 + p2 total density

#defining normalized drift speed using fitted density for vA
df_sf00['vdrift_va_pfits'] = df_sf00['vdrift']/df_sf00['valfven_pfits'] #using valfven from qtn

#defining center of mass velocity magnitude as |Vsw| compute Vsw mach
df_sf00['Vcm_mach_pfits'] = df_sf00['vcm_mag']/df_sf00['valfven_pfits'] #using valfven from fits
#defining p1 velocity magnitude as |Vsw| compute Vsw mach
df_sf00['Vp1_mach_pfits'] = df_sf00['vp1_mag']/df_sf00['valfven_pfits'] #using valfven from fits
#defining moment velocity magnitude as |Vsw| compute Vsw mach
df_sf00['vsw_mach_pfits'] = df_sf00['vsw_mom']/df_sf00['valfven_pfits'] #using valfven from fits

#compute proton beta 
#note 1e-5 is factor conversion from nT to cgs
df_sf00['beta_ppar_pfits'] = (4.03E-11 * df_sf00['n_tot'] * df_sf00['Tpar_tot'])/(1e-5 * df_sf00['B_mag'])**2 #total BetaPar from pfit dens
df_sf00['beta_pperp_pfits'] = (4.03E-11 * df_sf00['n_tot'] * df_sf00['Tperp_tot'])/(1e-5 * df_sf00['B_mag'])**2 #total BetaPar from pfit dens
df_sf00['beta_par_p1'] = (4.03E-11 * df_sf00['np1'] * df_sf00['Tpar1'])/(1e-5 * df_sf00['B_mag'])**2 #proton p1 BetaPar
df_sf00['beta_par_p2'] = (4.03E-11 * df_sf00['np2'] * df_sf00['Tpar2'])/(1e-5 * df_sf00['B_mag'])**2 #proton p2 BetaPar
df_sf00['beta_perp_p1'] = (4.03E-11 * df_sf00['np1'] * df_sf00['Tperp1'])/(1e-5 * df_sf00['B_mag'])**2 #proton p1 BetaPerp
df_sf00['beta_perp_p2'] = (4.03E-11 * df_sf00['np2'] * df_sf00['Tperp2'])/(1e-5 * df_sf00['B_mag'])**2 #proton p2 BetaPerp

df_sf00['beta_p_pfits_tot'] = np.sqrt(df_sf00['beta_ppar_pfits']**2 + df_sf00['beta_pperp_pfits']**2)

#hammerhead diagnostic
df_sf00['ham_param'] = (df_sf00['Tperp2']/df_sf00['Tperp1'])/df_sf00['Trat_tot']


#==============================================================
# If sQTN available, uncomment:
#==============================================================
#Access data from tplot vars
ne = get_data('electron_density')
te = get_data('electron_core_temperature')
#interpolate electron density and temperature to sf00 cadence
qtn_spi_t, ne_spi_up = resample(ne.y, ne.times, df_sf00.time)
qtn_spi_t, te_spi_up = resample(te.y, te.times, df_sf00.time)
#append to dataframe
df_sf00['n_e'] = ne_spi_up
df_sf00['t_e'] = te_spi_up
#calculate Alfven speed
df_sf00['valfven_qtn'] = 21.8 * df_sf00['B_mag']/ np.sqrt(df_sf00['n_e']) #using qtn density
#defining center of mass velocity magnitude as |Vsw| compute Vsw mach
df_sf00['Vcm_mach_qtn'] = df_sf00['vcm_mag']/df_sf00['valfven_qtn'] #using valfven from qtn
#defining p1 velocity magnitude as |Vsw| compute Vsw mach
df_sf00['Vp1_mach_qtn'] = df_sf00['vp1_mag']/df_sf00['valfven_qtn'] #using valfven from qtn
#defining moment velocity magnitude as |Vsw| compute Vsw mach
df_sf00['vsw_mach_qtn'] = df_sf00['vsw_mom']/df_sf00['valfven_qtn'] #using valfven from qtn
#defining normalized drift speed using qtn for vA
df_sf00['vdrift_va_qtn'] = df_sf00['vdrift']/df_sf00['valfven_qtn'] #using valfven from qtn
#calulate beta
df_sf00['beta_ppar_qtn'] = (4.03E-11 * df_sf00['n_e'] * df_sf00['Tpar_tot'])/(1e-5 * df_sf00['B_mag'])**2 #total BetaPar from qtn dens
df_sf00['beta_pperp_qtn'] = (4.03E-11 * df_sf00['n_e'] * df_sf00['Tperp_tot'])/(1e-5 * df_sf00['B_mag'])**2 #total BetaPerp from qtn dens
df_sf00['beta_p_qtn_tot'] = np.sqrt(df_sf00['beta_ppar_qtn']**2 + df_sf00['beta_pperp_qtn']**2)


Plot

In [ ]:

#adjust time range to plot
start = '2023-09-28/06:38'
stop = '2023-09-28/06:40'
xlim_datetime = [time_datetime(start), time_datetime(stop)]

#define order of variables on axes
ax_b = 0                    #magnetic field
ax_epad = 1                 #electron strahl pads
ax_spi_nrg = 2              #span energy flux vs energy
ax_spi_theta = 3            #span energy flux vs theta
ax_spi_phi = 4              #span energy flux vs phi
ax_densfits = 5             #density fits
ax_velfits = 6              #core velocity fits
ax_drift = 7                #beam-core drift velocity fits
ax_trat = 8                 #temperature anisotropy

#define smart y-limits for each variable
ylim_b = [np.min([np.min(br_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bt_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bn_4sa.y[time_clip(br_4sa.times, start, stop)])]) - 30 , np.max(bmag_rtn4sa[time_clip(br_4sa.times, start, stop)]) + 30 ]
ylim_densfits = [np.min([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],99.9), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],99.9)])]
#ylim_densfits = [np.min([np.min(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.min(df_sf00.np2[time_clip(df_sf00.time,start,stop)])]) - 10, np.max([np.max(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.max(df_sf00.np2[time_clip(df_sf00.time,start,stop)])])]
ylim_velfits =  [np.min([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],.15)]), np.max([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],99.6)])]
#ylim_velfits = [-500,500]
ylim_drift = [np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],.0001),np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],99.99)]
ylim_trat  =  [np.min([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],98)])]

#define smart z-limits for applicable variable
a = np.log10(epad_strahl[time_clip(epad_times,start,stop),:])
vmin_epad = np.round(np.nanmin(a[a != -np.inf])*10)/10   #round to nearest tenth #round to nearest tenth
vmax_epad = np.round(np.nanmax(np.log10(epad_strahl[time_clip(epad_times,start,stop),:]))*10)/10

plt.rcParams.update({'font.size': 32})

fig, ax = plt.subplots(9, 1, sharex=True, figsize=(30, 30),layout='compressed')
s=20
alpha=.7
linewidth = 2.0
pad = .003

ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bmag_rtn4sa[time_clip(br_4sa.times, start, stop)], label = '|B|', color='k', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),br_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Br', color='forestgreen', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bt_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bt', color='orange', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bn_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bn', color='dodgerblue', linewidth = linewidth)

s_epad=ax[ax_epad].pcolormesh(times_spe_repeat[time_clip(epad_times,start,stop),:],epad_PA_vals[time_clip(epad_times,start,stop),:],
         np.log10(epad_strahl[time_clip(epad_times,start,stop),:]),cmap='gist_stern', vmin = vmin_epad, vmax = vmax_epad)
cbar_epad = plt.colorbar(s_epad, pad = pad)
ax[ax_epad].plot(time_datetime(epad_times[time_clip(epad_times,start,stop)]),centroids_epad[time_clip(epad_times,start,stop)],color='black', linestyle='-', marker='o', markersize=4)

s_spi_nrg = ax[ax_spi_nrg].pcolormesh(times_spi_repeat[time_clip(spi_times,start,stop),:],spi_nrg_vals[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_nrg = plt.colorbar(s_spi_nrg, label=f'Flux ($eV/(cm^2 \\ s \\ sr \\ eV)$',pad = pad)

s_spi_theta = ax[ax_spi_theta].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_theta[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_theta[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_theta = plt.colorbar(s_spi_theta, pad = pad)

s_spi_phi = ax[ax_spi_phi].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_phi[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_phi[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_phi = plt.colorbar(s_spi_phi, pad = pad)
ax[ax_spi_phi].axhline(y = spi_nrg_vals_phi[0,1], color = 'red', linestyle = '--', linewidth = 3) #160
ax[ax_spi_phi].plot(time_datetime(spi_times[time_clip(spi_times,start,stop)]),centroids_spi_phi[time_clip(spi_times,start,stop)],color='black', linestyle='-', marker='o', markersize=2,linewidth = 2)

ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np1[time_clip(df_sf00.time,start,stop)], label = '$n_{p1}$', color = 'hotpink', marker=(5,1),s=s, alpha=alpha)
ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np2[time_clip(df_sf00.time,start,stop)], label = '$n_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)], label = '$vx_{p1}$', color = 'forestgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)], label = '$vy_{p1}$', color = 'orange', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)], label = '$vz_{p1}$', color = 'dodgerblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_drift].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat1[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p1}$', color = 'hotpink', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat2[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)], label = r'$T_\perp/T_\parallel$', color = 'mediumspringgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].axhline(y = 1, color = 'k', linestyle = '--', linewidth = 2) 

ax[ax_b].set(
    ylabel=f'$B$',
    ylim = ylim_b
    #xlim = xlim_datetime

)
ax[ax_epad].set(
    ylabel = r'Electron \n Strahl \n PAD ($^\circ$)'
)

ax[ax_spi_nrg].set(
    ylabel = 'Proton \n Energy \n Flux (eV)',
    ylim = [10,3e4]
)
ax[ax_spi_nrg].set_yscale('log')

ax[ax_spi_theta].set( 
     ylabel = 'Proton \n Eflux vs \n Theta (eV)',
     ylim = [-70,70]
)
ax[ax_spi_phi].set(
    ylabel = 'Proton \n Eflux vs \n  Phi (eV)',
    ylim = [80,190]
)

ax[ax_densfits].set(
    ylabel=f'N ($cm^{-3}$)',
    ylim = ylim_densfits
)

ax[ax_velfits].set(
    ylabel=r'Vel p1 (km/s)',
    ylim = ylim_velfits
)
ax[ax_drift].set(
    ylabel=f'Drift p2/vA',
    ylim = ylim_drift
)
ax[ax_trat].set(
    ylabel=r'$T_\perp/T_\parallel$',
    ylim = ylim_trat
)

#legends
l_b = ax[ax_b].legend(loc = 'upper right')
l_densfits = ax[ax_densfits].legend(loc = 'upper right')
l_densfits.legendHandles[0]._sizes = [200]
l_densfits.legendHandles[1]._sizes = [200]
l_velfits = ax[ax_velfits].legend(loc = 'upper right')
l_velfits.legendHandles[0]._sizes = [200]
l_velfits.legendHandles[1]._sizes = [200]
l_velfits.legendHandles[2]._sizes = [200]
l_drift = ax[ax_drift].legend(loc = 'upper right')
l_trat = ax[ax_trat].legend(loc = 'upper right')
l_trat.legendHandles[0]._sizes = [200]
l_trat.legendHandles[1]._sizes = [200]
l_trat.legendHandles[2]._sizes = [200]

plt.show()

In [ ]:

#adjust time range to plot
start = '2023-12-29/00:00'
stop = '2023-12-29/06:00'
xlim_datetime = [time_datetime(start), time_datetime(stop)]

#define order of variables on axes
ax_b = 0                    #magnetic field
ax_epad = 1                 #electron strahl pads
ax_spi_nrg = 2              #span energy flux vs energy
ax_spi_theta = 3            #span energy flux vs theta
ax_spi_phi = 4              #span energy flux vs phi
ax_densfits = 5             #density fits
ax_velfits = 6              #core velocity fits
ax_drift = 7                #beam-core drift velocity fits
ax_trat = 8                 #temperature anisotropy

#define smart y-limits for each variable
ylim_b = [np.min([np.min(br_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bt_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bn_4sa.y[time_clip(br_4sa.times, start, stop)])]) - 30 , np.max(bmag_rtn4sa[time_clip(br_4sa.times, start, stop)]) + 30 ]
ylim_densfits = [np.min([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],99.95), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],99.95)])]
#ylim_densfits = [np.min([np.min(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.min(df_sf00.np2[time_clip(df_sf00.time,start,stop)])]) - 10, np.max([np.max(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.max(df_sf00.np2[time_clip(df_sf00.time,start,stop)])])]
ylim_velfits =  [np.min([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],.15)]), np.max([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],99.6)])]
#ylim_velfits = [-500,500]
ylim_drift = [np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],.01),np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],99.96)]
ylim_trat  =  [np.min([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],98)])]

#define smart z-limits for applicable variable
a = np.log10(epad_strahl[time_clip(epad_times,start,stop),:])
vmin_epad = np.round(np.nanmin(a[a != -np.inf])*10)/10   #round to nearest tenth #round to nearest tenth
vmax_epad = np.round(np.nanmax(np.log10(epad_strahl[time_clip(epad_times,start,stop),:]))*10)/10

plt.rcParams.update({'font.size': 32})

fig, ax = plt.subplots(9, 1, sharex=True, figsize=(30, 30),layout='compressed')
s=20
alpha=.7
linewidth = 2.0
pad = .003

ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bmag_rtn4sa[time_clip(br_4sa.times, start, stop)], label = '|B|', color='k', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),br_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Br', color='forestgreen', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bt_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bt', color='orange', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bn_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bn', color='dodgerblue', linewidth = linewidth)

s_epad=ax[ax_epad].pcolormesh(times_spe_repeat[time_clip(epad_times,start,stop),:],epad_PA_vals[time_clip(epad_times,start,stop),:],
         np.log10(epad_strahl[time_clip(epad_times,start,stop),:]),cmap='gist_stern', vmin = vmin_epad, vmax = vmax_epad)
cbar_epad = plt.colorbar(s_epad, pad = pad)
ax[ax_epad].plot(time_datetime(epad_times[time_clip(epad_times,start,stop)]),centroids_epad[time_clip(epad_times,start,stop)],color='black', linestyle='-', marker='o', markersize=4)

s_spi_nrg = ax[ax_spi_nrg].pcolormesh(times_spi_repeat[time_clip(spi_times,start,stop),:],spi_nrg_vals[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_nrg = plt.colorbar(s_spi_nrg, label=f'Flux ($eV/(cm^2 \\ s \\ sr \\ eV)$',pad = pad)

s_spi_theta = ax[ax_spi_theta].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_theta[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_theta[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_theta = plt.colorbar(s_spi_theta, pad = pad)

s_spi_phi = ax[ax_spi_phi].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_phi[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_phi[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_phi = plt.colorbar(s_spi_phi, pad = pad)
ax[ax_spi_phi].axhline(y = spi_nrg_vals_phi[0,1], color = 'red', linestyle = '--', linewidth = 3) #160
ax[ax_spi_phi].plot(time_datetime(spi_times[time_clip(spi_times,start,stop)]),centroids_spi_phi[time_clip(spi_times,start,stop)],color='black', linestyle='-', marker='o', markersize=2,linewidth = 2)

ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np1[time_clip(df_sf00.time,start,stop)], label = '$n_{p1}$', color = 'hotpink', marker=(5,1),s=s, alpha=alpha)
ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np2[time_clip(df_sf00.time,start,stop)], label = '$n_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)], label = '$vx_{p1}$', color = 'forestgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)], label = '$vy_{p1}$', color = 'orange', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)], label = '$vz_{p1}$', color = 'dodgerblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_drift].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat1[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p1}$', color = 'hotpink', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat2[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)], label = r'$T_\perp/T_\parallel$', color = 'mediumspringgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].axhline(y = 1, color = 'k', linestyle = '--', linewidth = 2) 

ax[ax_b].set(
    ylabel=f'$B$',
    ylim = ylim_b
    #xlim = xlim_datetime

)
ax[ax_epad].set(
    ylabel = r'Electron \n Strahl \n PAD ($^\circ$)'
)

ax[ax_spi_nrg].set(
    ylabel = 'Proton \n Energy \n Flux (eV)',
    ylim = [10,3e4]
)
ax[ax_spi_nrg].set_yscale('log')

ax[ax_spi_theta].set( 
     ylabel = 'Proton \n Eflux vs \n Theta (eV)',
     ylim = [-70,70]
)
ax[ax_spi_phi].set(
    ylabel = 'Proton \n Eflux vs \n  Phi (eV)',
    ylim = [80,190]
)

ax[ax_densfits].set(
    ylabel=f'N ($cm^{-3}$)',
    ylim = ylim_densfits
)

ax[ax_velfits].set(
    ylabel=r'Vel p1 (km/s)',
    ylim = ylim_velfits
)
ax[ax_drift].set(
    ylabel=f'Drift p2/vA',
    ylim = ylim_drift
)
ax[ax_trat].set(
    ylabel=r'$T_\perp/T_\parallel$',
    ylim = ylim_trat
)

#legends
l_b = ax[ax_b].legend(loc = 'upper right')
l_densfits = ax[ax_densfits].legend(loc = 'upper right')
l_densfits.legendHandles[0]._sizes = [200]
l_densfits.legendHandles[1]._sizes = [200]
l_velfits = ax[ax_velfits].legend(loc = 'upper right')
l_velfits.legendHandles[0]._sizes = [200]
l_velfits.legendHandles[1]._sizes = [200]
l_velfits.legendHandles[2]._sizes = [200]
l_drift = ax[ax_drift].legend(loc = 'upper right')
l_trat = ax[ax_trat].legend(loc = 'upper right')
l_trat.legendHandles[0]._sizes = [200]
l_trat.legendHandles[1]._sizes = [200]
l_trat.legendHandles[2]._sizes = [200]

plt.show()

In [ ]:

#adjust time range to plot
start = '2023-12-29/01:30'
stop = '2023-12-29/03:30'
xlim_datetime = [time_datetime(start), time_datetime(stop)]

#define order of variables on axes
ax_b = 0                    #magnetic field
ax_epad = 1                 #electron strahl pads
ax_spi_nrg = 2              #span energy flux vs energy
ax_spi_theta = 3            #span energy flux vs theta
ax_spi_phi = 4              #span energy flux vs phi
ax_densfits = 5             #density fits
ax_velfits = 6              #core velocity fits
ax_drift = 7                #beam-core drift velocity fits
ax_trat = 8                 #temperature anisotropy

#define smart y-limits for each variable
ylim_b = [np.min([np.min(br_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bt_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bn_4sa.y[time_clip(br_4sa.times, start, stop)])]) - 30 , np.max(bmag_rtn4sa[time_clip(br_4sa.times, start, stop)]) + 30 ]
ylim_densfits = [np.min([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],99.9), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],99.9)])]
#ylim_densfits = [np.min([np.min(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.min(df_sf00.np2[time_clip(df_sf00.time,start,stop)])]) - 10, np.max([np.max(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.max(df_sf00.np2[time_clip(df_sf00.time,start,stop)])])]
ylim_velfits =  [np.min([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],.15)]), np.max([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],99.6)])]
#ylim_velfits = [-500,500]
ylim_drift = [np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],.05),np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],99.96)]
ylim_trat  =  [np.min([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],98)])]

#define smart z-limits for applicable variable
a = np.log10(epad_strahl[time_clip(epad_times,start,stop),:])
vmin_epad = np.round(np.nanmin(a[a != -np.inf])*10)/10   #round to nearest tenth #round to nearest tenth
vmax_epad = np.round(np.nanmax(np.log10(epad_strahl[time_clip(epad_times,start,stop),:]))*10)/10

plt.rcParams.update({'font.size': 32})

fig, ax = plt.subplots(9, 1, sharex=True, figsize=(30, 30),layout='compressed')
s=20
alpha=.7
linewidth = 2.0
pad = .003

ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bmag_rtn4sa[time_clip(br_4sa.times, start, stop)], label = '|B|', color='k', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),br_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Br', color='forestgreen', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bt_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bt', color='orange', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bn_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bn', color='dodgerblue', linewidth = linewidth)

s_epad=ax[ax_epad].pcolormesh(times_spe_repeat[time_clip(epad_times,start,stop),:],epad_PA_vals[time_clip(epad_times,start,stop),:],
         np.log10(epad_strahl[time_clip(epad_times,start,stop),:]),cmap='gist_stern', vmin = vmin_epad, vmax = vmax_epad)
cbar_epad = plt.colorbar(s_epad, pad = pad)
ax[ax_epad].plot(time_datetime(epad_times[time_clip(epad_times,start,stop)]),centroids_epad[time_clip(epad_times,start,stop)],color='black', linestyle='-', marker='o', markersize=4)

s_spi_nrg = ax[ax_spi_nrg].pcolormesh(times_spi_repeat[time_clip(spi_times,start,stop),:],spi_nrg_vals[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_nrg = plt.colorbar(s_spi_nrg, label=f'Flux ($eV/(cm^2 \\ s \\ sr \\ eV)$',pad = pad)

s_spi_theta = ax[ax_spi_theta].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_theta[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_theta[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_theta = plt.colorbar(s_spi_theta, pad = pad)

s_spi_phi = ax[ax_spi_phi].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_phi[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_phi[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_phi = plt.colorbar(s_spi_phi, pad = pad)
ax[ax_spi_phi].axhline(y = spi_nrg_vals_phi[0,1], color = 'red', linestyle = '--', linewidth = 3) #160
ax[ax_spi_phi].plot(time_datetime(spi_times[time_clip(spi_times,start,stop)]),centroids_spi_phi[time_clip(spi_times,start,stop)],color='black', linestyle='-', marker='o', markersize=2,linewidth = 2)

ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np1[time_clip(df_sf00.time,start,stop)], label = '$n_{p1}$', color = 'hotpink', marker=(5,1),s=s, alpha=alpha)
ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np2[time_clip(df_sf00.time,start,stop)], label = '$n_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)], label = '$vx_{p1}$', color = 'forestgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)], label = '$vy_{p1}$', color = 'orange', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)], label = '$vz_{p1}$', color = 'dodgerblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_drift].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat1[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p1}$', color = 'hotpink', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat2[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)], label = r'$T_\perp/T_\parallel$', color = 'mediumspringgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].axhline(y = 1, color = 'k', linestyle = '--', linewidth = 2) 

ax[ax_b].set(
    ylabel=f'$B$',
    ylim = ylim_b
    #xlim = xlim_datetime

)
ax[ax_epad].set(
    ylabel = f'Electron \n Strahl \n PAD' + r'($^\circ$)'
)

ax[ax_spi_nrg].set(
    ylabel = 'Proton \n Energy \n Flux (eV)',
    ylim = [10,3e4]
)
ax[ax_spi_nrg].set_yscale('log')

ax[ax_spi_theta].set( 
     ylabel = 'Proton \n Eflux vs \n Theta (eV)',
     ylim = [-70,70]
)
ax[ax_spi_phi].set(
    ylabel = 'Proton \n Eflux vs \n  Phi (eV)',
    ylim = [80,190]
)

ax[ax_densfits].set(
    ylabel=f'N ($cm^{-3}$)',
    ylim = ylim_densfits
)

ax[ax_velfits].set(
    ylabel=r'Vel p1 (km/s)',
    ylim = ylim_velfits
)
ax[ax_drift].set(
    ylabel=f'Drift p2/vA',
    ylim = ylim_drift
)
ax[ax_trat].set(
    ylabel=r'$T_\perp/T_\parallel$',
    ylim = ylim_trat
)

#legends
l_b = ax[ax_b].legend(loc = 'upper right')
l_densfits = ax[ax_densfits].legend(loc = 'upper right')
l_densfits.legendHandles[0]._sizes = [200]
l_densfits.legendHandles[1]._sizes = [200]
l_velfits = ax[ax_velfits].legend(loc = 'upper right')
l_velfits.legendHandles[0]._sizes = [200]
l_velfits.legendHandles[1]._sizes = [200]
l_velfits.legendHandles[2]._sizes = [200]
l_drift = ax[ax_drift].legend(loc = 'upper right')
l_trat = ax[ax_trat].legend(loc = 'upper right')
l_trat.legendHandles[0]._sizes = [200]
l_trat.legendHandles[1]._sizes = [200]
l_trat.legendHandles[2]._sizes = [200]

plt.show()

In [ ]:

#adjust time range to plot
start = '2023-12-29/01:30'
stop = '2023-12-29/03:30'
xlim_datetime = [time_datetime(start), time_datetime(stop)]

#define order of variables on axes
ax_b = 0                    #magnetic field
ax_epad = 1                 #electron strahl pads
ax_spi_nrg = 2              #span energy flux vs energy
ax_spi_theta = 3            #span energy flux vs theta
ax_spi_phi = 4              #span energy flux vs phi
ax_densfits = 5             #density fits
ax_velfits = 6              #core velocity fits
ax_drift = 7                #beam-core drift velocity fits
ax_trat = 8                 #temperature anisotropy

#define smart y-limits for each variable
ylim_b = [np.min([np.min(br_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bt_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bn_4sa.y[time_clip(br_4sa.times, start, stop)])]) - 30 , np.max(bmag_rtn4sa[time_clip(br_4sa.times, start, stop)]) + 30 ]
ylim_densfits = [np.min([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],99.9), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],99.9)])]
#ylim_densfits = [np.min([np.min(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.min(df_sf00.np2[time_clip(df_sf00.time,start,stop)])]) - 10, np.max([np.max(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.max(df_sf00.np2[time_clip(df_sf00.time,start,stop)])])]
ylim_velfits =  [np.min([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],.15)]), np.max([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],99.6)])]
#ylim_velfits = [-500,500]
ylim_drift = [np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],.05),np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],99.96)]
ylim_trat  =  [np.min([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],98)])]

#define smart z-limits for applicable variable
a = np.log10(epad_strahl[time_clip(epad_times,start,stop),:])
vmin_epad = np.round(np.nanmin(a[a != -np.inf])*10)/10   #round to nearest tenth #round to nearest tenth
vmax_epad = np.round(np.nanmax(np.log10(epad_strahl[time_clip(epad_times,start,stop),:]))*10)/10

plt.rcParams.update({'font.size': 32})

fig, ax = plt.subplots(9, 1, sharex=True, figsize=(30, 30),layout='compressed')
s=20
alpha=.7
linewidth = 2.0
pad = .003

ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bmag_rtn4sa[time_clip(br_4sa.times, start, stop)], label = '|B|', color='k', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),br_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Br', color='forestgreen', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bt_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bt', color='orange', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bn_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bn', color='dodgerblue', linewidth = linewidth)

s_epad=ax[ax_epad].pcolormesh(times_spe_repeat[time_clip(epad_times,start,stop),:],epad_PA_vals[time_clip(epad_times,start,stop),:],
         np.log10(epad_strahl[time_clip(epad_times,start,stop),:]),cmap='gist_stern', vmin = vmin_epad, vmax = vmax_epad)
cbar_epad = plt.colorbar(s_epad, pad = pad)
ax[ax_epad].plot(time_datetime(epad_times[time_clip(epad_times,start,stop)]),centroids_epad[time_clip(epad_times,start,stop)],color='black', linestyle='-', marker='o', markersize=4)

s_spi_nrg = ax[ax_spi_nrg].pcolormesh(times_spi_repeat[time_clip(spi_times,start,stop),:],spi_nrg_vals[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_nrg = plt.colorbar(s_spi_nrg, label=f'Flux ($eV/(cm^2 \\ s \\ sr \\ eV)$',pad = pad)

s_spi_theta = ax[ax_spi_theta].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_theta[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_theta[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_theta = plt.colorbar(s_spi_theta, pad = pad)

s_spi_phi = ax[ax_spi_phi].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_phi[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_phi[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_phi = plt.colorbar(s_spi_phi, pad = pad)
ax[ax_spi_phi].axhline(y = spi_nrg_vals_phi[0,1], color = 'red', linestyle = '--', linewidth = 3) #160
ax[ax_spi_phi].plot(time_datetime(spi_times[time_clip(spi_times,start,stop)]),centroids_spi_phi[time_clip(spi_times,start,stop)],color='black', linestyle='-', marker='o', markersize=2,linewidth = 2)

ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np1[time_clip(df_sf00.time,start,stop)], label = '$n_{p1}$', color = 'hotpink', marker=(5,1),s=s, alpha=alpha)
ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np2[time_clip(df_sf00.time,start,stop)], label = '$n_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)], label = '$vx_{p1}$', color = 'forestgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)], label = '$vy_{p1}$', color = 'orange', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)], label = '$vz_{p1}$', color = 'dodgerblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_drift].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat1[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p1}$', color = 'hotpink', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat2[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)], label = r'$T_\perp/T_\parallel$', color = 'mediumspringgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].axhline(y = 1, color = 'k', linestyle = '--', linewidth = 2) 

ax[ax_b].set(
    ylabel=f'$B$',
    ylim = ylim_b
    #xlim = xlim_datetime

)
ax[ax_epad].set(
    ylabel = f'Electron \n Strahl \n PAD' + r'($^\circ$)'
)

ax[ax_spi_nrg].set(
    ylabel = 'Proton \n Energy \n Flux (eV)',
    ylim = [10,3e4]
)
ax[ax_spi_nrg].set_yscale('log')

ax[ax_spi_theta].set( 
     ylabel = 'Proton \n Eflux vs \n Theta (eV)',
     ylim = [-70,70]
)
ax[ax_spi_phi].set(
    ylabel = 'Proton \n Eflux vs \n  Phi (eV)',
    ylim = [80,190]
)

ax[ax_densfits].set(
    ylabel=f'N ($cm^{-3}$)',
    ylim = ylim_densfits
)

ax[ax_velfits].set(
    ylabel=r'Vel p1 (km/s)',
    ylim = ylim_velfits
)
ax[ax_drift].set(
    ylabel=f'Drift p2/vA',
    ylim = ylim_drift
)
ax[ax_trat].set(
    ylabel=r'$T_\perp/T_\parallel$',
    ylim = ylim_trat
)
ax[ax_trat].set_yscale('log')

#legends
l_b = ax[ax_b].legend(loc = 'upper right')
l_densfits = ax[ax_densfits].legend(loc = 'upper right')
l_densfits.legendHandles[0]._sizes = [200]
l_densfits.legendHandles[1]._sizes = [200]
l_velfits = ax[ax_velfits].legend(loc = 'upper right')
l_velfits.legendHandles[0]._sizes = [200]
l_velfits.legendHandles[1]._sizes = [200]
l_velfits.legendHandles[2]._sizes = [200]
l_drift = ax[ax_drift].legend(loc = 'upper right')
l_trat = ax[ax_trat].legend(loc = 'upper right')
l_trat.legendHandles[0]._sizes = [200]
l_trat.legendHandles[1]._sizes = [200]
l_trat.legendHandles[2]._sizes = [200]

plt.show()

In [ ]:

#adjust time range to plot
start = '2023-12-29/01:54'
stop = '2023-12-29/02:00'
xlim_datetime = [time_datetime(start), time_datetime(stop)]

#define order of variables on axes
ax_b = 0                    #magnetic field
ax_epad = 1                 #electron strahl pads
ax_spi_nrg = 2              #span energy flux vs energy
ax_spi_theta = 3            #span energy flux vs theta
ax_spi_phi = 4              #span energy flux vs phi
ax_densfits = 5             #density fits
ax_velfits = 6              #core velocity fits
ax_drift = 7                #beam-core drift velocity fits
ax_trat = 8                 #temperature anisotropy

#define smart y-limits for each variable
ylim_b = [np.min([np.min(br_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bt_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bn_4sa.y[time_clip(br_4sa.times, start, stop)])]) - 30 , np.max(bmag_rtn4sa[time_clip(br_4sa.times, start, stop)]) + 30 ]
ylim_densfits = [np.min([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],99.9), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],99.9)])]
#ylim_densfits = [np.min([np.min(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.min(df_sf00.np2[time_clip(df_sf00.time,start,stop)])]) - 10, np.max([np.max(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.max(df_sf00.np2[time_clip(df_sf00.time,start,stop)])])]
ylim_velfits =  [np.min([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],.15)]), np.max([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],99.6)])]
#ylim_velfits = [-500,500]
ylim_drift = [np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],.05),np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],99.96)]
ylim_trat  =  [np.min([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],98)])]

#define smart z-limits for applicable variable
a = np.log10(epad_strahl[time_clip(epad_times,start,stop),:])
vmin_epad = np.round(np.nanmin(a[a != -np.inf])*10)/10   #round to nearest tenth #round to nearest tenth
vmax_epad = np.round(np.nanmax(np.log10(epad_strahl[time_clip(epad_times,start,stop),:]))*10)/10

plt.rcParams.update({'font.size': 32})

fig, ax = plt.subplots(9, 1, sharex=True, figsize=(30, 30),layout='compressed')
s=200
alpha=1
linewidth = 3.0
pad = .003

ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bmag_rtn4sa[time_clip(br_4sa.times, start, stop)], label = '|B|', color='k', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),br_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Br', color='forestgreen', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bt_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bt', color='orange', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bn_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bn', color='dodgerblue', linewidth = linewidth)

s_epad=ax[ax_epad].pcolormesh(times_spe_repeat[time_clip(epad_times,start,stop),:],epad_PA_vals[time_clip(epad_times,start,stop),:],
         np.log10(epad_strahl[time_clip(epad_times,start,stop),:]),cmap='gist_stern', vmin = vmin_epad, vmax = vmax_epad)
cbar_epad = plt.colorbar(s_epad, pad = pad)
ax[ax_epad].plot(time_datetime(epad_times[time_clip(epad_times,start,stop)]),centroids_epad[time_clip(epad_times,start,stop)],color='black', linestyle='-', marker='o', markersize=4)

s_spi_nrg = ax[ax_spi_nrg].pcolormesh(times_spi_repeat[time_clip(spi_times,start,stop),:],spi_nrg_vals[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_nrg = plt.colorbar(s_spi_nrg, label=f'Flux ($eV/(cm^2 \\ s \\ sr \\ eV)$',pad = pad)

s_spi_theta = ax[ax_spi_theta].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_theta[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_theta[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_theta = plt.colorbar(s_spi_theta, pad = pad)

s_spi_phi = ax[ax_spi_phi].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_phi[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_phi[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_phi = plt.colorbar(s_spi_phi, pad = pad)
ax[ax_spi_phi].axhline(y = spi_nrg_vals_phi[0,1], color = 'red', linestyle = '--', linewidth = 3) #160
ax[ax_spi_phi].plot(time_datetime(spi_times[time_clip(spi_times,start,stop)]),centroids_spi_phi[time_clip(spi_times,start,stop)],color='black', linestyle='-', marker='o', markersize=2,linewidth = 2)

ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np1[time_clip(df_sf00.time,start,stop)], label = '$n_{p1}$', color = 'hotpink', marker=(5,1),s=s, alpha=alpha)
ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np2[time_clip(df_sf00.time,start,stop)], label = '$n_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)], label = '$vx_{p1}$', color = 'forestgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)], label = '$vy_{p1}$', color = 'orange', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)], label = '$vz_{p1}$', color = 'dodgerblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_drift].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat1[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p1}$', color = 'hotpink', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat2[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)], label = r'$T_\perp/T_\parallel$', color = 'mediumspringgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].axhline(y = 1, color = 'k', linestyle = '--', linewidth = 2) 

ax[ax_b].set(
    ylabel=f'$B$',
    ylim = ylim_b
    #xlim = xlim_datetime

)
ax[ax_epad].set(
    ylabel = f'Electron \n Strahl \n PAD' + r'($^\circ$)'
)

ax[ax_spi_nrg].set(
    ylabel = 'Proton \n Energy \n Flux (eV)',
    ylim = [10,1e5]
)
ax[ax_spi_nrg].set_yscale('log')

ax[ax_spi_theta].set( 
     ylabel = 'Proton \n Eflux vs \n Theta (eV)',
     ylim = [-70,70]
)
ax[ax_spi_phi].set(
    ylabel = 'Proton \n Eflux vs \n  Phi (eV)',
    ylim = [80,190]
)

ax[ax_densfits].set(
    ylabel=f'N ($cm^{-3}$)',
   # ylim = ylim_densfits
)

ax[ax_velfits].set(
    ylabel=r'Vel p1 (km/s)',
   # ylim = ylim_velfits
)
ax[ax_drift].set(
    ylabel=f'Drift p2/vA',
 #   ylim = ylim_drift
)
ax[ax_trat].set(
    ylabel=r'$T_\perp/T_\parallel$',
  #  ylim = ylim_trat
)
ax[ax_trat].set_yscale('log')

#legends
l_b = ax[ax_b].legend(loc = 'upper right')
l_densfits = ax[ax_densfits].legend(loc = 'upper right')
l_densfits.legendHandles[0]._sizes = [200]
l_densfits.legendHandles[1]._sizes = [200]
l_velfits = ax[ax_velfits].legend(loc = 'upper right')
l_velfits.legendHandles[0]._sizes = [200]
l_velfits.legendHandles[1]._sizes = [200]
l_velfits.legendHandles[2]._sizes = [200]
l_drift = ax[ax_drift].legend(loc = 'upper right')
l_trat = ax[ax_trat].legend(loc = 'upper right')
l_trat.legendHandles[0]._sizes = [200]
l_trat.legendHandles[1]._sizes = [200]
l_trat.legendHandles[2]._sizes = [200]

plt.show()

In [ ]:

#adjust time range to plot
start = '2023-12-29/01:56'
stop = '2023-12-29/01:57'
xlim_datetime = [time_datetime(start), time_datetime(stop)]

#define order of variables on axes
ax_b = 0                    #magnetic field
ax_epad = 1                 #electron strahl pads
ax_spi_nrg = 2              #span energy flux vs energy
ax_spi_theta = 3            #span energy flux vs theta
ax_spi_phi = 4              #span energy flux vs phi
ax_densfits = 5             #density fits
ax_velfits = 6              #core velocity fits
ax_drift = 7                #beam-core drift velocity fits
ax_trat = 8                 #temperature anisotropy

#define smart y-limits for each variable
ylim_b = [np.min([np.min(br_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bt_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bn_4sa.y[time_clip(br_4sa.times, start, stop)])]) - 30 , np.max(bmag_rtn4sa[time_clip(br_4sa.times, start, stop)]) + 30 ]
ylim_densfits = [np.min([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],99.9), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],99.9)])]
#ylim_densfits = [np.min([np.min(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.min(df_sf00.np2[time_clip(df_sf00.time,start,stop)])]) - 10, np.max([np.max(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.max(df_sf00.np2[time_clip(df_sf00.time,start,stop)])])]
ylim_velfits =  [np.min([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],.15)]), np.max([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],99.6)])]
#ylim_velfits = [-500,500]
ylim_drift = [np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],.05),np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],99.96)]
ylim_trat  =  [np.min([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],98)])]

#define smart z-limits for applicable variable
a = np.log10(epad_strahl[time_clip(epad_times,start,stop),:])
vmin_epad = np.round(np.nanmin(a[a != -np.inf])*10)/10   #round to nearest tenth #round to nearest tenth
vmax_epad = np.round(np.nanmax(np.log10(epad_strahl[time_clip(epad_times,start,stop),:]))*10)/10

plt.rcParams.update({'font.size': 32})

fig, ax = plt.subplots(9, 1, sharex=True, figsize=(30, 30),layout='compressed')
s=200
alpha=1
linewidth = 3.0
pad = .003

ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bmag_rtn4sa[time_clip(br_4sa.times, start, stop)], label = '|B|', color='k', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),br_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Br', color='forestgreen', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bt_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bt', color='orange', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bn_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bn', color='dodgerblue', linewidth = linewidth)

s_epad=ax[ax_epad].pcolormesh(times_spe_repeat[time_clip(epad_times,start,stop),:],epad_PA_vals[time_clip(epad_times,start,stop),:],
         np.log10(epad_strahl[time_clip(epad_times,start,stop),:]),cmap='gist_stern', vmin = vmin_epad, vmax = vmax_epad)
cbar_epad = plt.colorbar(s_epad, pad = pad)
ax[ax_epad].plot(time_datetime(epad_times[time_clip(epad_times,start,stop)]),centroids_epad[time_clip(epad_times,start,stop)],color='black', linestyle='-', marker='o', markersize=4)

s_spi_nrg = ax[ax_spi_nrg].pcolormesh(times_spi_repeat[time_clip(spi_times,start,stop),:],spi_nrg_vals[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_nrg = plt.colorbar(s_spi_nrg, label=f'Flux ($eV/(cm^2 \\ s \\ sr \\ eV)$',pad = pad)

s_spi_theta = ax[ax_spi_theta].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_theta[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_theta[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_theta = plt.colorbar(s_spi_theta, pad = pad)

s_spi_phi = ax[ax_spi_phi].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_phi[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_phi[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_phi = plt.colorbar(s_spi_phi, pad = pad)
ax[ax_spi_phi].axhline(y = spi_nrg_vals_phi[0,1], color = 'red', linestyle = '--', linewidth = 3) #160
ax[ax_spi_phi].plot(time_datetime(spi_times[time_clip(spi_times,start,stop)]),centroids_spi_phi[time_clip(spi_times,start,stop)],color='black', linestyle='-', marker='o', markersize=2,linewidth = 2)

ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np1[time_clip(df_sf00.time,start,stop)], label = '$n_{p1}$', color = 'hotpink', marker=(5,1),s=s, alpha=alpha)
ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np2[time_clip(df_sf00.time,start,stop)], label = '$n_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)], label = '$vx_{p1}$', color = 'forestgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)], label = '$vy_{p1}$', color = 'orange', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)], label = '$vz_{p1}$', color = 'dodgerblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_drift].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat1[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p1}$', color = 'hotpink', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat2[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)], label = r'$T_\perp/T_\parallel$', color = 'mediumspringgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].axhline(y = 1, color = 'k', linestyle = '--', linewidth = 2) 

ax[ax_b].set(
    ylabel=f'$B$',
    ylim = ylim_b
    #xlim = xlim_datetime

)
ax[ax_epad].set(
    ylabel = f'Electron \n Strahl \n PAD' + r'($^\circ$)'
)

ax[ax_spi_nrg].set(
    ylabel = 'Proton \n Energy \n Flux (eV)',
    ylim = [10,1e5]
)
ax[ax_spi_nrg].set_yscale('log')

ax[ax_spi_theta].set( 
     ylabel = 'Proton \n Eflux vs \n Theta (eV)',
     ylim = [-70,70]
)
ax[ax_spi_phi].set(
    ylabel = 'Proton \n Eflux vs \n  Phi (eV)',
    ylim = [80,190]
)

ax[ax_densfits].set(
    ylabel=f'N ($cm^{-3}$)',
   # ylim = ylim_densfits
)

ax[ax_velfits].set(
    ylabel=r'Vel p1 (km/s)',
   # ylim = ylim_velfits
)
ax[ax_drift].set(
    ylabel=f'Drift p2/vA',
 #   ylim = ylim_drift
)
ax[ax_trat].set(
    ylabel=r'$T_\perp/T_\parallel$',
  #  ylim = ylim_trat
)
ax[ax_trat].set_yscale('log')

#legends
l_b = ax[ax_b].legend(loc = 'upper right')
l_densfits = ax[ax_densfits].legend(loc = 'upper right')
l_densfits.legendHandles[0]._sizes = [200]
l_densfits.legendHandles[1]._sizes = [200]
l_velfits = ax[ax_velfits].legend(loc = 'upper right')
l_velfits.legendHandles[0]._sizes = [200]
l_velfits.legendHandles[1]._sizes = [200]
l_velfits.legendHandles[2]._sizes = [200]
l_drift = ax[ax_drift].legend(loc = 'upper right')
l_trat = ax[ax_trat].legend(loc = 'upper right')
l_trat.legendHandles[0]._sizes = [200]
l_trat.legendHandles[1]._sizes = [200]
l_trat.legendHandles[2]._sizes = [200]

plt.show()

In [ ]:
start = '2023-12-29/01:56:10'
stop = '2023-12-29/01:56:11'
ind = time_clip(df_sf00.time,start,stop)
print('time = ',df_sf00[ind].datetime.values )
print('np1 = ', df_sf00[ind].np1.values)
print('np2 = ', df_sf00[ind].np2.values)
print('(vp1_x, vp1_y, vp2_z) = ', df_sf00[ind].vp1_x.values,df_sf00[ind].vp1_y.values ,df_sf00[ind].vp1_z.values)
print('vp2_drift = ', df_sf00[ind].vdrift.values)
print('temp1 = ', df_sf00[ind].Tperp1.values)
print('temp2 = ', df_sf00[ind].Tperp2.values)
print('trat1 = ', df_sf00[ind].Trat1.values)
print('trat2 = ', df_sf00[ind].Trat2.values)

In [ ]:
start = '2023-12-29/01:56:10'
stop = '2023-12-29/01:56:11'
df_sf00_tclip = convert_time(start,stop)
print(df_sf00_tclip.np1)

In [ ]:
##### #adjust time range to plot
start = '2023-12-29/02:00'
stop = '2023-12-29/03:00'
xlim_datetime = [time_datetime(start), time_datetime(stop)]

#define order of variables on axes
ax_b = 0                    #magnetic field
ax_epad = 1                 #electron strahl pads
ax_spi_nrg = 2              #span energy flux vs energy
ax_spi_theta = 3            #span energy flux vs theta
ax_spi_phi = 4              #span energy flux vs phi
ax_densfits = 5             #density fits
ax_velfits = 6              #core velocity fits
ax_drift = 7                #beam-core drift velocity fits
ax_trat = 8                 #temperature anisotropy

#define smart y-limits for each variable
ylim_b = [np.min([np.min(br_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bt_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bn_4sa.y[time_clip(br_4sa.times, start, stop)])]) - 30 , np.max(bmag_rtn4sa[time_clip(br_4sa.times, start, stop)]) + 30 ]
ylim_densfits = [np.min([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],99.9), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],99.9)])]
#ylim_densfits = [np.min([np.min(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.min(df_sf00.np2[time_clip(df_sf00.time,start,stop)])]) - 10, np.max([np.max(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.max(df_sf00.np2[time_clip(df_sf00.time,start,stop)])])]
ylim_velfits =  [np.min([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],.15)]), np.max([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],99.6)])]
#ylim_velfits = [-500,500]
ylim_drift = [np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],.05),np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],99.96)]
ylim_trat  =  [np.min([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],98)])]

#define smart z-limits for applicable variable
a = np.log10(epad_strahl[time_clip(epad_times,start,stop),:])
vmin_epad = np.round(np.nanmin(a[a != -np.inf])*10)/10   #round to nearest tenth #round to nearest tenth
vmax_epad = np.round(np.nanmax(np.log10(epad_strahl[time_clip(epad_times,start,stop),:]))*10)/10

plt.rcParams.update({'font.size': 32})

fig, ax = plt.subplots(9, 1, sharex=True, figsize=(30, 30),layout='compressed')
s=20
alpha=.7
linewidth = 2.0
pad = .003

ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bmag_rtn4sa[time_clip(br_4sa.times, start, stop)], label = '|B|', color='k', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),br_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Br', color='forestgreen', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bt_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bt', color='orange', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bn_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bn', color='dodgerblue', linewidth = linewidth)

s_epad=ax[ax_epad].pcolormesh(times_spe_repeat[time_clip(epad_times,start,stop),:],epad_PA_vals[time_clip(epad_times,start,stop),:],
         np.log10(epad_strahl[time_clip(epad_times,start,stop),:]),cmap='gist_stern', vmin = vmin_epad, vmax = vmax_epad)
cbar_epad = plt.colorbar(s_epad, pad = pad)
ax[ax_epad].plot(time_datetime(epad_times[time_clip(epad_times,start,stop)]),centroids_epad[time_clip(epad_times,start,stop)],color='black', linestyle='-', marker='o', markersize=4)

s_spi_nrg = ax[ax_spi_nrg].pcolormesh(times_spi_repeat[time_clip(spi_times,start,stop),:],spi_nrg_vals[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_nrg = plt.colorbar(s_spi_nrg, label=f'Flux ($eV/(cm^2 \\ s \\ sr \\ eV)$',pad = pad)

s_spi_theta = ax[ax_spi_theta].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_theta[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_theta[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_theta = plt.colorbar(s_spi_theta, pad = pad)

s_spi_phi = ax[ax_spi_phi].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_phi[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_phi[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_phi = plt.colorbar(s_spi_phi, pad = pad)
ax[ax_spi_phi].axhline(y = spi_nrg_vals_phi[0,1], color = 'red', linestyle = '--', linewidth = 3) #160
ax[ax_spi_phi].plot(time_datetime(spi_times[time_clip(spi_times,start,stop)]),centroids_spi_phi[time_clip(spi_times,start,stop)],color='black', linestyle='-', marker='o', markersize=2,linewidth = 2)

ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np1[time_clip(df_sf00.time,start,stop)], label = '$n_{p1}$', color = 'hotpink', marker=(5,1),s=s, alpha=alpha)
ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np2[time_clip(df_sf00.time,start,stop)], label = '$n_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)], label = '$vx_{p1}$', color = 'forestgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)], label = '$vy_{p1}$', color = 'orange', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)], label = '$vz_{p1}$', color = 'dodgerblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_drift].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat1[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p1}$', color = 'hotpink', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat2[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)], label = r'$T_\perp/T_\parallel$', color = 'mediumspringgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].axhline(y = 1, color = 'k', linestyle = '--', linewidth = 2) 

ax[ax_b].set(
    ylabel=f'$B$',
    ylim = ylim_b
    #xlim = xlim_datetime

)
ax[ax_epad].set(
    ylabel = r'Electron \n Strahl \n PAD ($^\circ$)'
)

ax[ax_spi_nrg].set(
    ylabel = 'Proton \n Energy \n Flux (eV)',
    ylim = [10,3e4]
)
ax[ax_spi_nrg].set_yscale('log')

ax[ax_spi_theta].set( 
     ylabel = 'Proton \n Eflux vs \n Theta (eV)',
     ylim = [-70,70]
)
ax[ax_spi_phi].set(
    ylabel = 'Proton \n Eflux vs \n  Phi (eV)',
    ylim = [80,190]
)

ax[ax_densfits].set(
    ylabel=f'N ($cm^{-3}$)',
    ylim = ylim_densfits
)

ax[ax_velfits].set(
    ylabel=r'Vel p1 (km/s)',
    ylim = ylim_velfits
)
ax[ax_drift].set(
    ylabel=f'Drift p2/vA',
    ylim = ylim_drift
)
ax[ax_trat].set(
    ylabel=r'$T_\perp/T_\parallel$',
    ylim = ylim_trat
)

#legends
l_b = ax[ax_b].legend(loc = 'upper right')
l_densfits = ax[ax_densfits].legend(loc = 'upper right')
l_densfits.legendHandles[0]._sizes = [200]
l_densfits.legendHandles[1]._sizes = [200]
l_velfits = ax[ax_velfits].legend(loc = 'upper right')
l_velfits.legendHandles[0]._sizes = [200]
l_velfits.legendHandles[1]._sizes = [200]
l_velfits.legendHandles[2]._sizes = [200]
l_drift = ax[ax_drift].legend(loc = 'upper right')
l_trat = ax[ax_trat].legend(loc = 'upper right')
l_trat.legendHandles[0]._sizes = [200]
l_trat.legendHandles[1]._sizes = [200]
l_trat.legendHandles[2]._sizes = [200]

plt.show()

In [ ]:

#adjust time range to plot
start = '2023-12-29/02:40'
stop = '2023-12-29/02:51'
xlim_datetime = [time_datetime(start), time_datetime(stop)]

#define order of variables on axes
ax_b = 0                    #magnetic field
ax_epad = 1                 #electron strahl pads
ax_spi_nrg = 2              #span energy flux vs energy
ax_spi_theta = 3            #span energy flux vs theta
ax_spi_phi = 4              #span energy flux vs phi
ax_densfits = 5             #density fits
ax_velfits = 6              #core velocity fits
ax_drift = 7                #beam-core drift velocity fits
ax_trat = 8                 #temperature anisotropy

#define smart y-limits for each variable
ylim_b = [np.min([np.min(br_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bt_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bn_4sa.y[time_clip(br_4sa.times, start, stop)])]) - 30 , np.max(bmag_rtn4sa[time_clip(br_4sa.times, start, stop)]) + 30 ]
ylim_densfits = [np.min([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],99.9), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],99.9)])]
#ylim_densfits = [np.min([np.min(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.min(df_sf00.np2[time_clip(df_sf00.time,start,stop)])]) - 10, np.max([np.max(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.max(df_sf00.np2[time_clip(df_sf00.time,start,stop)])])]
ylim_velfits =  [np.min([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],.15)]), np.max([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],99.6)])]
#ylim_velfits = [-500,500]
ylim_drift = [np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],.05),np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],99.96)]
ylim_trat  =  [np.min([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],98)])]

#define smart z-limits for applicable variable
a = np.log10(epad_strahl[time_clip(epad_times,start,stop),:])
vmin_epad = np.round(np.nanmin(a[a != -np.inf])*10)/10   #round to nearest tenth #round to nearest tenth
vmax_epad = np.round(np.nanmax(np.log10(epad_strahl[time_clip(epad_times,start,stop),:]))*10)/10

plt.rcParams.update({'font.size': 32})

fig, ax = plt.subplots(9, 1, sharex=True, figsize=(30, 30),layout='compressed')
s=200
alpha=1
linewidth = 3.0
pad = .003

ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bmag_rtn4sa[time_clip(br_4sa.times, start, stop)], label = '|B|', color='k', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),br_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Br', color='forestgreen', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bt_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bt', color='orange', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bn_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bn', color='dodgerblue', linewidth = linewidth)

s_epad=ax[ax_epad].pcolormesh(times_spe_repeat[time_clip(epad_times,start,stop),:],epad_PA_vals[time_clip(epad_times,start,stop),:],
         np.log10(epad_strahl[time_clip(epad_times,start,stop),:]),cmap='gist_stern', vmin = vmin_epad, vmax = vmax_epad)
cbar_epad = plt.colorbar(s_epad, pad = pad)
ax[ax_epad].plot(time_datetime(epad_times[time_clip(epad_times,start,stop)]),centroids_epad[time_clip(epad_times,start,stop)],color='black', linestyle='-', marker='o', markersize=4)

s_spi_nrg = ax[ax_spi_nrg].pcolormesh(times_spi_repeat[time_clip(spi_times,start,stop),:],spi_nrg_vals[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_nrg = plt.colorbar(s_spi_nrg, label=f'Flux ($eV/(cm^2 \\ s \\ sr \\ eV)$',pad = pad)

s_spi_theta = ax[ax_spi_theta].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_theta[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_theta[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_theta = plt.colorbar(s_spi_theta, pad = pad)

s_spi_phi = ax[ax_spi_phi].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_phi[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_phi[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_phi = plt.colorbar(s_spi_phi, pad = pad)
ax[ax_spi_phi].axhline(y = spi_nrg_vals_phi[0,1], color = 'red', linestyle = '--', linewidth = 3) #160
ax[ax_spi_phi].plot(time_datetime(spi_times[time_clip(spi_times,start,stop)]),centroids_spi_phi[time_clip(spi_times,start,stop)],color='black', linestyle='-', marker='o', markersize=2,linewidth = 2)

ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np1[time_clip(df_sf00.time,start,stop)], label = '$n_{p1}$', color = 'hotpink', marker=(5,1),s=s, alpha=alpha)
ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np2[time_clip(df_sf00.time,start,stop)], label = '$n_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)], label = '$vx_{p1}$', color = 'forestgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)], label = '$vy_{p1}$', color = 'orange', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)], label = '$vz_{p1}$', color = 'dodgerblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_drift].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat1[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p1}$', color = 'hotpink', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat2[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)], label = r'$T_\perp/T_\parallel$', color = 'mediumspringgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].axhline(y = 1, color = 'k', linestyle = '--', linewidth = 2) 

ax[ax_b].set(
    ylabel=f'$B$',
    ylim = ylim_b
    #xlim = xlim_datetime

)
ax[ax_epad].set(
    ylabel = r'Electron \n Strahl \n PAD ($^\circ$)'
)

ax[ax_spi_nrg].set(
    ylabel = 'Proton \n Energy \n Flux (eV)',
    ylim = [10,3e4]
)
ax[ax_spi_nrg].set_yscale('log')

ax[ax_spi_theta].set( 
     ylabel = 'Proton \n Eflux vs \n Theta (eV)',
     ylim = [-70,70]
)
ax[ax_spi_phi].set(
    ylabel = 'Proton \n Eflux vs \n  Phi (eV)',
    ylim = [80,190]
)

ax[ax_densfits].set(
    ylabel=f'N ($cm^{-3}$)',
    ylim = ylim_densfits
)

ax[ax_velfits].set(
    ylabel=r'Vel p1 (km/s)',
    ylim = ylim_velfits
)
ax[ax_drift].set(
    ylabel=f'Drift p2/vA',
    ylim = ylim_drift
)
ax[ax_trat].set(
    ylabel=r'$T_\perp/T_\parallel$',
    ylim = ylim_trat
)

#legends
l_b = ax[ax_b].legend(loc = 'upper right')
l_densfits = ax[ax_densfits].legend(loc = 'upper right')
l_densfits.legendHandles[0]._sizes = [200]
l_densfits.legendHandles[1]._sizes = [200]
l_velfits = ax[ax_velfits].legend(loc = 'upper right')
l_velfits.legendHandles[0]._sizes = [200]
l_velfits.legendHandles[1]._sizes = [200]
l_velfits.legendHandles[2]._sizes = [200]
l_drift = ax[ax_drift].legend(loc = 'upper right')
l_trat = ax[ax_trat].legend(loc = 'upper right')
l_trat.legendHandles[0]._sizes = [200]
l_trat.legendHandles[1]._sizes = [200]
l_trat.legendHandles[2]._sizes = [200]

plt.show()

In [ ]:

#adjust time range to plot
start = '2023-12-29/02:42:50'
stop = '2023-12-29/02:50'
xlim_datetime = [time_datetime(start), time_datetime(stop)]

#define order of variables on axes
ax_b = 0                    #magnetic field
ax_epad = 1                 #electron strahl pads
ax_spi_nrg = 2              #span energy flux vs energy
ax_spi_theta = 3            #span energy flux vs theta
ax_spi_phi = 4              #span energy flux vs phi
ax_densfits = 5             #density fits
ax_velfits = 6              #core velocity fits
ax_drift = 7                #beam-core drift velocity fits
ax_trat = 8                 #temperature anisotropy

#define smart y-limits for each variable
ylim_b = [np.min([np.min(br_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bt_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bn_4sa.y[time_clip(br_4sa.times, start, stop)])]) - 30 , np.max(bmag_rtn4sa[time_clip(br_4sa.times, start, stop)]) + 30 ]
ylim_densfits = [np.min([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],99.9), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],99.9)])]
#ylim_densfits = [np.min([np.min(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.min(df_sf00.np2[time_clip(df_sf00.time,start,stop)])]) - 10, np.max([np.max(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.max(df_sf00.np2[time_clip(df_sf00.time,start,stop)])])]
ylim_velfits =  [np.min([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],.15)]), np.max([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],99.6)])]
#ylim_velfits = [-500,500]
ylim_drift = [np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],.05),np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],99.96)]
ylim_trat  =  [np.min([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],98)])]

#define smart z-limits for applicable variable
a = np.log10(epad_strahl[time_clip(epad_times,start,stop),:])
vmin_epad = np.round(np.nanmin(a[a != -np.inf])*10)/10   #round to nearest tenth #round to nearest tenth
vmax_epad = np.round(np.nanmax(np.log10(epad_strahl[time_clip(epad_times,start,stop),:]))*10)/10

plt.rcParams.update({'font.size': 32})

fig, ax = plt.subplots(9, 1, sharex=True, figsize=(30, 30),layout='compressed')
s=200
alpha=1
linewidth = 3.0
pad = .003

ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bmag_rtn4sa[time_clip(br_4sa.times, start, stop)], label = '|B|', color='k', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),br_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Br', color='forestgreen', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bt_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bt', color='orange', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bn_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bn', color='dodgerblue', linewidth = linewidth)

s_epad=ax[ax_epad].pcolormesh(times_spe_repeat[time_clip(epad_times,start,stop),:],epad_PA_vals[time_clip(epad_times,start,stop),:],
         np.log10(epad_strahl[time_clip(epad_times,start,stop),:]),cmap='gist_stern', vmin = vmin_epad, vmax = vmax_epad)
cbar_epad = plt.colorbar(s_epad, pad = pad)
ax[ax_epad].plot(time_datetime(epad_times[time_clip(epad_times,start,stop)]),centroids_epad[time_clip(epad_times,start,stop)],color='black', linestyle='-', marker='o', markersize=4)

s_spi_nrg = ax[ax_spi_nrg].pcolormesh(times_spi_repeat[time_clip(spi_times,start,stop),:],spi_nrg_vals[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_nrg = plt.colorbar(s_spi_nrg, label=f'Flux ($eV/(cm^2 \\ s \\ sr \\ eV)$',pad = pad)

s_spi_theta = ax[ax_spi_theta].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_theta[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_theta[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_theta = plt.colorbar(s_spi_theta, pad = pad)

s_spi_phi = ax[ax_spi_phi].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_phi[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_phi[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_phi = plt.colorbar(s_spi_phi, pad = pad)
ax[ax_spi_phi].axhline(y = spi_nrg_vals_phi[0,1], color = 'red', linestyle = '--', linewidth = 3) #160
ax[ax_spi_phi].plot(time_datetime(spi_times[time_clip(spi_times,start,stop)]),centroids_spi_phi[time_clip(spi_times,start,stop)],color='black', linestyle='-', marker='o', markersize=2,linewidth = 2)

ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np1[time_clip(df_sf00.time,start,stop)], label = '$n_{p1}$', color = 'hotpink', marker=(5,1),s=s, alpha=alpha)
ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np2[time_clip(df_sf00.time,start,stop)], label = '$n_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)], label = '$vx_{p1}$', color = 'forestgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)], label = '$vy_{p1}$', color = 'orange', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)], label = '$vz_{p1}$', color = 'dodgerblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_drift].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat1[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p1}$', color = 'hotpink', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat2[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)], label = r'$T_\perp/T_\parallel$', color = 'mediumspringgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].axhline(y = 1, color = 'k', linestyle = '--', linewidth = 2) 

ax[ax_b].set(
    ylabel=f'$B$',
    ylim = ylim_b
    #xlim = xlim_datetime

)
ax[ax_epad].set(
    ylabel =  
)

ax[ax_spi_nrg].set(
    ylabel = 'Proton \n Energy \n Flux (eV)',
    ylim = [10,3e4]
)
ax[ax_spi_nrg].set_yscale('log')

ax[ax_spi_theta].set( 
     ylabel = 'Proton \n Eflux vs \n Theta (eV)',
     ylim = [-70,70]
)
ax[ax_spi_phi].set(
    ylabel = 'Proton \n Eflux vs \n  Phi (eV)',
    ylim = [80,190]
)

ax[ax_densfits].set(
    ylabel=f'N ($cm^{-3}$)',
 #   ylim = ylim_densfits
)

ax[ax_velfits].set(
    ylabel=r'Vel p1 (km/s)',
   # ylim = ylim_velfits
)
ax[ax_drift].set(
    ylabel=f'Drift p2/vA',
   # ylim = ylim_drift
)
ax[ax_trat].set(
    ylabel=r'$T_\perp/T_\parallel$',
 #   ylim = ylim_trat
)

#legends
l_b = ax[ax_b].legend(loc = 'upper right')
l_densfits = ax[ax_densfits].legend(loc = 'upper right')
l_densfits.legendHandles[0]._sizes = [200]
l_densfits.legendHandles[1]._sizes = [200]
l_velfits = ax[ax_velfits].legend(loc = 'upper right')
l_velfits.legendHandles[0]._sizes = [200]
l_velfits.legendHandles[1]._sizes = [200]
l_velfits.legendHandles[2]._sizes = [200]
l_drift = ax[ax_drift].legend(loc = 'upper right')
l_trat = ax[ax_trat].legend(loc = 'upper right')
l_trat.legendHandles[0]._sizes = [200]
l_trat.legendHandles[1]._sizes = [200]
l_trat.legendHandles[2]._sizes = [200]

plt.show()

In [ ]:

#adjust time range to plot
start = '2023-12-29/02:43:50'
stop = '2023-12-29/02:44:50'
xlim_datetime = [time_datetime(start), time_datetime(stop)]

#define order of variables on axes
ax_b = 0                    #magnetic field
ax_epad = 1                 #electron strahl pads
ax_spi_nrg = 2              #span energy flux vs energy
ax_spi_theta = 3            #span energy flux vs theta
ax_spi_phi = 4              #span energy flux vs phi
ax_densfits = 5             #density fits
ax_velfits = 6              #core velocity fits
ax_drift = 7                #beam-core drift velocity fits
ax_trat = 8                 #temperature anisotropy

#define smart y-limits for each variable
ylim_b = [np.min([np.min(br_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bt_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bn_4sa.y[time_clip(br_4sa.times, start, stop)])]) - 30 , np.max(bmag_rtn4sa[time_clip(br_4sa.times, start, stop)]) + 30 ]
ylim_densfits = [np.min([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],99.9), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],99.9)])]
#ylim_densfits = [np.min([np.min(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.min(df_sf00.np2[time_clip(df_sf00.time,start,stop)])]) - 10, np.max([np.max(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.max(df_sf00.np2[time_clip(df_sf00.time,start,stop)])])]
ylim_velfits =  [np.min([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],.15)]), np.max([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],99.6)])]
#ylim_velfits = [-500,500]
ylim_drift = [np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],.05),np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],99.96)]
ylim_trat  =  [np.min([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],98)])]

#define smart z-limits for applicable variable
a = np.log10(epad_strahl[time_clip(epad_times,start,stop),:])
vmin_epad = np.round(np.nanmin(a[a != -np.inf])*10)/10   #round to nearest tenth #round to nearest tenth
vmax_epad = np.round(np.nanmax(np.log10(epad_strahl[time_clip(epad_times,start,stop),:]))*10)/10

plt.rcParams.update({'font.size': 32})

fig, ax = plt.subplots(9, 1, sharex=True, figsize=(30, 30),layout='compressed')
s=200
alpha=1
linewidth = 3.0
pad = .003

ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bmag_rtn4sa[time_clip(br_4sa.times, start, stop)], label = '|B|', color='k', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),br_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Br', color='forestgreen', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bt_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bt', color='orange', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bn_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bn', color='dodgerblue', linewidth = linewidth)

s_epad=ax[ax_epad].pcolormesh(times_spe_repeat[time_clip(epad_times,start,stop),:],epad_PA_vals[time_clip(epad_times,start,stop),:],
         np.log10(epad_strahl[time_clip(epad_times,start,stop),:]),cmap='gist_stern', vmin = vmin_epad, vmax = vmax_epad)
cbar_epad = plt.colorbar(s_epad, pad = pad)
ax[ax_epad].plot(time_datetime(epad_times[time_clip(epad_times,start,stop)]),centroids_epad[time_clip(epad_times,start,stop)],color='black', linestyle='-', marker='o', markersize=4)

s_spi_nrg = ax[ax_spi_nrg].pcolormesh(times_spi_repeat[time_clip(spi_times,start,stop),:],spi_nrg_vals[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_nrg = plt.colorbar(s_spi_nrg, label=f'Flux ($eV/(cm^2 \\ s \\ sr \\ eV)$',pad = pad)

s_spi_theta = ax[ax_spi_theta].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_theta[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_theta[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_theta = plt.colorbar(s_spi_theta, pad = pad)

s_spi_phi = ax[ax_spi_phi].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_phi[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_phi[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_phi = plt.colorbar(s_spi_phi, pad = pad)
ax[ax_spi_phi].axhline(y = spi_nrg_vals_phi[0,1], color = 'red', linestyle = '--', linewidth = 3) #160
ax[ax_spi_phi].plot(time_datetime(spi_times[time_clip(spi_times,start,stop)]),centroids_spi_phi[time_clip(spi_times,start,stop)],color='black', linestyle='-', marker='o', markersize=2,linewidth = 2)

ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np1[time_clip(df_sf00.time,start,stop)], label = '$n_{p1}$', color = 'hotpink', marker=(5,1),s=s, alpha=alpha)
ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np2[time_clip(df_sf00.time,start,stop)], label = '$n_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)], label = '$vx_{p1}$', color = 'forestgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)], label = '$vy_{p1}$', color = 'orange', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)], label = '$vz_{p1}$', color = 'dodgerblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_drift].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat1[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p1}$', color = 'hotpink', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat2[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)], label = r'$T_\perp/T_\parallel$', color = 'mediumspringgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].axhline(y = 1, color = 'k', linestyle = '--', linewidth = 2) 

ax[ax_b].set(
    ylabel=f'$B$',
  #  ylim = ylim_b
    #xlim = xlim_datetime

)
ax[ax_epad].set(
    ylabel = f'Electron \n Strahl \n PAD' + r'($^\circ$)'
)

ax[ax_spi_nrg].set(
    ylabel = 'Proton \n Energy \n Flux (eV)',
    ylim = [10,3e4]
)
ax[ax_spi_nrg].set_yscale('log')

ax[ax_spi_theta].set( 
     ylabel = 'Proton \n Eflux vs \n Theta (eV)',
     ylim = [-70,70]
)
ax[ax_spi_phi].set(
    ylabel = 'Proton \n Eflux vs \n  Phi (eV)',
    ylim = [80,190]
)

ax[ax_densfits].set(
    ylabel=f'N ($cm^{-3}$)',
 #   ylim = ylim_densfits
)

ax[ax_velfits].set(
    ylabel=r'Vel p1 (km/s)',
   # ylim = ylim_velfits
)
ax[ax_drift].set(
    ylabel=f'Drift p2/vA',
   # ylim = ylim_drift
)
ax[ax_trat].set(
    ylabel=r'$T_\perp/T_\parallel$',
 #   ylim = ylim_trat
)

#legends
l_b = ax[ax_b].legend(loc = 'upper right')
l_densfits = ax[ax_densfits].legend(loc = 'upper right')
l_densfits.legendHandles[0]._sizes = [200]
l_densfits.legendHandles[1]._sizes = [200]
l_velfits = ax[ax_velfits].legend(loc = 'upper right')
l_velfits.legendHandles[0]._sizes = [200]
l_velfits.legendHandles[1]._sizes = [200]
l_velfits.legendHandles[2]._sizes = [200]
l_drift = ax[ax_drift].legend(loc = 'upper right')
l_trat = ax[ax_trat].legend(loc = 'upper right')
l_trat.legendHandles[0]._sizes = [200]
l_trat.legendHandles[1]._sizes = [200]
l_trat.legendHandles[2]._sizes = [200]

plt.show()

In [ ]:

#adjust time range to plot
start = '2023-12-29/02:47:50'
stop = '2023-12-29/02:50:00'
xlim_datetime = [time_datetime(start), time_datetime(stop)]

#define order of variables on axes
ax_b = 0                    #magnetic field
ax_epad = 1                 #electron strahl pads
ax_spi_nrg = 2              #span energy flux vs energy
ax_spi_theta = 3            #span energy flux vs theta
ax_spi_phi = 4              #span energy flux vs phi
ax_densfits = 5             #density fits
ax_velfits = 6              #core velocity fits
ax_drift = 7                #beam-core drift velocity fits
ax_trat = 8                 #temperature anisotropy

#define smart y-limits for each variable
ylim_b = [np.min([np.min(br_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bt_4sa.y[time_clip(br_4sa.times, start, stop)]), np.min(bn_4sa.y[time_clip(br_4sa.times, start, stop)])]) - 30 , np.max(bmag_rtn4sa[time_clip(br_4sa.times, start, stop)]) + 30 ]
ylim_densfits = [np.min([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.np1[time_clip(df_sf00.time,start,stop)],99.9), np.nanpercentile(df_sf00.np2[time_clip(df_sf00.time,start,stop)],99.9)])]
#ylim_densfits = [np.min([np.min(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.min(df_sf00.np2[time_clip(df_sf00.time,start,stop)])]) - 10, np.max([np.max(df_sf00.np1[time_clip(df_sf00.time,start,stop)]), np.max(df_sf00.np2[time_clip(df_sf00.time,start,stop)])])]
ylim_velfits =  [np.min([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],.15), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],.15)]), np.max([np.nanpercentile(df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)],99.6), np.nanpercentile(df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)],99.6)])]
#ylim_velfits = [-500,500]
ylim_drift = [np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],.05),np.nanpercentile(df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)],99.96)]
ylim_trat  =  [np.min([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],.01), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],.01)]), np.max([np.nanpercentile(df_sf00.Trat1[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat2[time_clip(df_sf00.time,start,stop)],98), np.nanpercentile(df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)],98)])]

#define smart z-limits for applicable variable
a = np.log10(epad_strahl[time_clip(epad_times,start,stop),:])
vmin_epad = np.round(np.nanmin(a[a != -np.inf])*10)/10   #round to nearest tenth #round to nearest tenth
vmax_epad = np.round(np.nanmax(np.log10(epad_strahl[time_clip(epad_times,start,stop),:]))*10)/10

plt.rcParams.update({'font.size': 32})

fig, ax = plt.subplots(9, 1, sharex=True, figsize=(30, 30),layout='compressed')
s=200
alpha=1
linewidth = 3.0
pad = .003

ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bmag_rtn4sa[time_clip(br_4sa.times, start, stop)], label = '|B|', color='k', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),br_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Br', color='forestgreen', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bt_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bt', color='orange', linewidth = linewidth)
ax[ax_b].plot(time_datetime(br_4sa.times[time_clip(br_4sa.times, start, stop)]),bn_4sa.y[time_clip(br_4sa.times, start, stop)], label = 'Bn', color='dodgerblue', linewidth = linewidth)

s_epad=ax[ax_epad].pcolormesh(times_spe_repeat[time_clip(epad_times,start,stop),:],epad_PA_vals[time_clip(epad_times,start,stop),:],
         np.log10(epad_strahl[time_clip(epad_times,start,stop),:]),cmap='gist_stern', vmin = vmin_epad, vmax = vmax_epad)
cbar_epad = plt.colorbar(s_epad, pad = pad)
ax[ax_epad].plot(time_datetime(epad_times[time_clip(epad_times,start,stop)]),centroids_epad[time_clip(epad_times,start,stop)],color='black', linestyle='-', marker='o', markersize=4)

s_spi_nrg = ax[ax_spi_nrg].pcolormesh(times_spi_repeat[time_clip(spi_times,start,stop),:],spi_nrg_vals[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_nrg = plt.colorbar(s_spi_nrg, label=f'Flux ($eV/(cm^2 \\ s \\ sr \\ eV)$',pad = pad)

s_spi_theta = ax[ax_spi_theta].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_theta[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_theta[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_theta = plt.colorbar(s_spi_theta, pad = pad)

s_spi_phi = ax[ax_spi_phi].pcolormesh(times_spi_repeat_angle[time_clip(spi_times,start,stop),:],spi_nrg_vals_phi[time_clip(spi_times,start,stop),:],
         np.log10(spi_nrg_flux_phi[time_clip(spi_times,start,stop),:]),cmap='terrain')
cbar_spi_phi = plt.colorbar(s_spi_phi, pad = pad)
ax[ax_spi_phi].axhline(y = spi_nrg_vals_phi[0,1], color = 'red', linestyle = '--', linewidth = 3) #160
ax[ax_spi_phi].plot(time_datetime(spi_times[time_clip(spi_times,start,stop)]),centroids_spi_phi[time_clip(spi_times,start,stop)],color='black', linestyle='-', marker='o', markersize=2,linewidth = 2)

ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np1[time_clip(df_sf00.time,start,stop)], label = '$n_{p1}$', color = 'hotpink', marker=(5,1),s=s, alpha=alpha)
ax[ax_densfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.np2[time_clip(df_sf00.time,start,stop)], label = '$n_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_x[time_clip(df_sf00.time,start,stop)], label = '$vx_{p1}$', color = 'forestgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_y[time_clip(df_sf00.time,start,stop)], label = '$vy_{p1}$', color = 'orange', marker=(5,1), s=s, alpha=alpha)
ax[ax_velfits].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vp1_z[time_clip(df_sf00.time,start,stop)], label = '$vz_{p1}$', color = 'dodgerblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_drift].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vdrift_va_pfits[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat1[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p1}$', color = 'hotpink', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat2[time_clip(df_sf00.time,start,stop)], label = r'$\frac{T_\perp}{T_\parallel}_{p2}$', color = 'deepskyblue', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.Trat_tot[time_clip(df_sf00.time,start,stop)], label = r'$T_\perp/T_\parallel$', color = 'mediumspringgreen', marker=(5,1), s=s, alpha=alpha)
ax[ax_trat].axhline(y = 1, color = 'k', linestyle = '--', linewidth = 2) 

ax[ax_b].set(
    ylabel=f'$B$',
  #  ylim = ylim_b
    #xlim = xlim_datetime

)
ax[ax_epad].set(
    ylabel = f'Electron \n Strahl \n PAD' + r'($^\circ$)'
)

ax[ax_spi_nrg].set(
    ylabel = 'Proton \n Energy \n Flux (eV)',
    ylim = [10,3e4]
)
ax[ax_spi_nrg].set_yscale('log')

ax[ax_spi_theta].set( 
     ylabel = 'Proton \n Eflux vs \n Theta (eV)',
     ylim = [-70,70]
)
ax[ax_spi_phi].set(
    ylabel = 'Proton \n Eflux vs \n  Phi (eV)',
    ylim = [80,190]
)

ax[ax_densfits].set(
    ylabel=f'N ($cm^{-3}$)',
 #   ylim = ylim_densfits
)

ax[ax_velfits].set(
    ylabel=r'Vel p1 (km/s)',
   # ylim = ylim_velfits
)
ax[ax_drift].set(
    ylabel=f'Drift p2/vA',
   # ylim = ylim_drift
)
ax[ax_trat].set(
    ylabel=r'$T_\perp/T_\parallel$',
 #   ylim = ylim_trat
)

#legends
l_b = ax[ax_b].legend(loc = 'upper right')
l_densfits = ax[ax_densfits].legend(loc = 'upper right')
l_densfits.legendHandles[0]._sizes = [200]
l_densfits.legendHandles[1]._sizes = [200]
l_velfits = ax[ax_velfits].legend(loc = 'upper right')
l_velfits.legendHandles[0]._sizes = [200]
l_velfits.legendHandles[1]._sizes = [200]
l_velfits.legendHandles[2]._sizes = [200]
l_drift = ax[ax_drift].legend(loc = 'upper right')
l_trat = ax[ax_trat].legend(loc = 'upper right')
l_trat.legendHandles[0]._sizes = [200]
l_trat.legendHandles[1]._sizes = [200]
l_trat.legendHandles[2]._sizes = [200]

plt.show()

In [ ]:
a = np.log10(epad_strahl[time_clip(epad_times,start,stop),:])
print(np.nanmin(a[a != -np.inf]))
print(np.nanmax(np.log10(epad_strahl[time_clip(epad_times,start,stop),:])))

In [ ]:
np.round(10.98*10)/10

In [ ]:
plt.scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.vdrift[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)


In [ ]:
plt.scatter(df_sf00.datetime[time_clip(df_sf00.time,start,stop)], df_sf00.n_e[time_clip(df_sf00.time,start,stop)], label = '$vdrift_{p2}/vA$', color = 'navy', marker=(5,1), s=s, alpha=alpha)
